In [7]:
import pandas as pd
import numpy as np
from langdetect import detect
import re

from openpyxl import Workbook

import os
import time
import datetime

In [8]:
from bs4 import BeautifulSoup as bs
import json

In [9]:
import glob

In [10]:
# Python3 program to Grouping list 
# elements based on frequency
from collections import Counter
import itertools

In [11]:
from googletrans import Translator, constants

In [28]:
# streamlit app modules
import streamlit as st 
import sklearn
import plotly
import matplotlib

In [30]:
print('--Python module versions')

print(f'pandas=={pd.__version__}')
print(f'numpy=={np.__version__}')
print(f're=={re.__version__}')
print(f'json=={json.__version__}')

print('--Streamlit app module versions')

print(f'streamlit=={st.__version__}')
print(f'sklearn=={sklearn.__version__}')
print(f'plotly=={plotly.__version__}')
print(f'matplotlib=={matplotlib.__version__}')

--Python module versions
pandas==2.1.2
numpy==1.26.2
re==2.2.1
json==2.0.9
--Streamlit app module versions
streamlit==1.31.0
sklearn==1.3.2
plotly==5.16.1
matplotlib==3.8.2


In [7]:
# Look for keyword in a dictionary and return the dictionary key that contains the keyword
def diveDictValues(keyword = None, dict_to_search = {}):
    if keyword is None:
        return None
    if dict_to_search is False:
        return None
    
    for key in dict_to_search.keys():
        try:
            if str(dict_to_search[key]).__contains__(keyword):
                return key
        except KeyError:
            continue

    return None

# Find keyword in a dictionary or list of strings
def diveListValues(keyword = None, list_to_search = []):
    if keyword is None:
        return None
    if list_to_search is False:
        return None
    
    matched_items = []
    for i in range(len(list_to_search)):
        if type(list_to_search[i]) == dict:
            if keyword in list(list_to_search[i].keys()):
                matched_items.append(i)
        elif str(list_to_search[i]).__contains__(keyword):
            matched_items.append(i)

    return matched_items

# Get instructions to iterate deeper through a nested json text file and perform certain instructions if nest level is a dictionary or list
def ProcessHtmlText(dict_to_search = None, dict_labels = []):
    if dict_to_search is None:
        return None
    
    curr_dict_level = dict_to_search
    result = None

    try:
        for i in range(len(dict_labels)):
            label = dict_labels[i]
            if label.__contains__('list'):
                if type(curr_dict_level) == list:
                    dive_result = diveListValues(label.split('-')[1], curr_dict_level)
                    if dive_result is None:
                        continue
                    elif len(dive_result) <= 0:
                        continue
                    elif len(dive_result) > 1:
                        possible_nests_to_check = {}
                        for j in dive_result:
                            possible_nests_to_check[j] = curr_dict_level[j]
                        accepted_nests = AttemptPossibleNest(possible_nests_to_check, dict_labels[i+1:])
                        dive_result = accepted_nests[-1]
                    elif len(dive_result) == 1:
                        dive_result = dive_result[0]
 
                    curr_dict_level = curr_dict_level[dive_result]
            else:
                if type(curr_dict_level) == dict:
                    try:
                        # dive_result = diveDictValues(label, curr_dict_level)
                        # if dive_result is None:
                        #     continue
                        
                        # print(f"label: {label}; result: {dive_result}")
                        curr_dict_level = curr_dict_level[label]
                    except KeyError:
                        continue

            if label == dict_labels[-1]:
                result = curr_dict_level
    except ValueError:
        return None

    return result

# To get ahead of the arbitrary header names, attempt the neighbouring nest level to look for the interested keyword
# Similar to a search binary tree, search through the node of neighbours to find the interested keyword
def AttemptPossibleNest(possible_nest_dict = {}, next_labels = []):
    accepted_nests = []
    for key in possible_nest_dict.keys():
        nest_to_dive = possible_nest_dict[key]
        
        # print('nesting', key, next_labels, nest_to_dive)
        result_check = ProcessHtmlText(nest_to_dive, dict_labels=next_labels)
        if result_check is not None:
            accepted_nests.append(key)
    
    return accepted_nests

# Parse HTML text into XML and iterate through ids to get the expected value
def diveHtmlTags(html_body, toplevel_tag = None, dict_labels = [], multi = False, find_keyword = None):
    soup = bs(html_body, 'lxml')
    if toplevel_tag is None:
        return None
    if dict_labels is False:
        return None
    
    ids = [ids['id'] for ids in soup.find_all(toplevel_tag, id=True)]
    
    result = None
    if multi == False:
        for id_key in ids:
            long_text = soup.find(id = id_key).text
            try:
                json_dict = json.loads(long_text, strict = False)

                if json_dict is None:
                    continue
                
                result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
                if result_check is not None:
                    if find_keyword is None:
                        result = result_check
                    elif result_check.__contains__(find_keyword):
                        # print('Found "employees"')
                        # print(result_check)
                        # print('--')
                        result = result_check
                    else:
                        result = result_check     


            except ValueError:  # includes simplejson.decoder.JSONDecodeError
                # print(f'Decoding JSON has failed at {id_key}')
                continue
    else:
        result = []
        for id_key in ids:
            long_text = soup.find(id = id_key).text
            try:
                json_dict = json.loads(long_text, strict = False)

                if json_dict is None:
                    continue
                
                result_check = ProcessHtmlText(json_dict, dict_labels=dict_labels)
                if result_check is not None:
                    print('rc')
                    print(f"id: {id_key}")
                    print(result_check)
                    print('--')
                    result.append(result_check)
            except ValueError:  # includes simplejson.decoder.JSONDecodeError
                print(f'Decoding JSON has failed at {id_key}')
                continue

    return result


In [8]:
# Get Job Post ID
def GetJobPostingID(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['jobPostingId']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'jobPostingId'])
    except:
        return None

# Get Job Post Status
def GetPostStatus(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['jobState']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'jobState'])
    except:
        return None

# Get Link of Job Post on LinkedIn
def GetPostLink(url_link = None, job_id = None):
    try:
        if url_link is not None:
            return url_link
        if job_id is not None:
            return f'https://www.linkedin.com/jobs/view/{job_id}/'
    except:
        return None
    
    return None

# Get Job Title
def GetJobTitle(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['title']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'title'])
    except:
        return None

# Get Job Description
def GetJobDescription(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['description']['text']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'description', 'text'])
    except:
        return None

# Get the language of the Job Description
def GetDescriptionLanguage(jd_text):
    try:
        return detect(jd_text)
    except:
        return ''

# Get Job Remote Option
def GetRemoteWorkStatus(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['workRemoteAllowed']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'workRemoteAllowed'])
    except:
        return None

# Get Job Reposted Status
def GetRepostStatus(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['repostedJobPosting']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'repostedJobPosting'])
    except:
        return None

# Get Job Location
def GetJobCountry(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['formattedLocation']
        return diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'formattedLocation'])
    except:
        return None

# Get Job Top Card Details
# Example: ['Remote', 'Full-time', 'Entry level']
def GetJobTopCardDetails(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912217
        # id_key = 'bpr-guid-15912217'
        # long_text = soup.find(id = id_key)
        # json_dict = json.loads(long_text.text, strict = False)
        # return [t['text']['text'] for t in json_dict['included'][4]['jobInsightsV2ResolutionResults'][0]['jobInsightViewModel']['description']]
        dive_result = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults','list-jobInsightViewModel', 'jobInsightViewModel', 'description'])
        # print('dr type', type(dive_result))
        # print(dive_result.keys())
        # print(dive_result)
        # print([t.keys() for t in dive_result])
        return [t['text']['text'] for t in dive_result]
    except:
        return None

# Get Experience Level
# Example: ['Internship', 'Entry', 'Associate', 'Mid-Senior', 'Director', 'Executive']
def GetJobExperienceLevel(html_body, details = []):
    try:
        if details is True:
            for keyword in ['Internship', 'Entry', 'Associate', 'Mid-Senior', 'Director', 'Executive']:
                for detail in details:
                    if keyword in detail:
                        return detail
                    
        details = GetJobTopCardDetails(html_body)
        for keyword in ['Internship', 'Entry', 'Associate', 'Mid-Senior', 'Director', 'Executive']:
            for detail in details:
                if keyword in detail:
                    return detail
    except:
        return None
    
    return None

# Get Work Setting Placement
# Example: ['Hybrid', 'Remote', 'On-site']
def GetJobWorkSetting(html_body, details = []):
    try:
        if details is True:
            for keyword in ['Hybrid', 'Remote', 'On-site']:
                for detail in details:
                    if keyword in detail:
                        return detail
                    
        details = GetJobTopCardDetails(html_body)
        for keyword in ['Hybrid', 'Remote', 'On-site']:
            for detail in details:
                if keyword in detail:
                    return detail
    except:
        return None
    
    return None

# Get Employment Type
# Example: ['Full-time', 'Part-time', 'Contract', 'Temporary', 'Volunteer', 'Internship', 'Other']
def GetJobType(html_body, details = []):
    try:
        if details is True:
            for keyword in ['Full-time', 'Part-time', 'Contract', 'Temporary', 'Volunteer', 'Internship', 'Other']:
                for detail in details:
                    if keyword in detail:
                        return detail
                    
        details = GetJobTopCardDetails(html_body)
        for keyword in ['Full-time', 'Part-time', 'Contract', 'Temporary', 'Volunteer', 'Internship', 'Other']:
            for detail in details:
                if keyword in detail:
                    return detail
    except:
        return None
    
    return None

# Get Skills Listed on Job Post
def GetJobSkills(html_body):
    try:
        soup = bs(html_body)

        return [li.find('div').get_text().strip() for li in soup.find('ul', {'class': 'job-details-skill-match-status-list'}).find_all('li')]
    except:
        return None

def GetJobYearsExperience(html_body, job_des = None, des_lan = 'en'):
    try:
        def GetYears_RegEx(des):
            years_text = re.findall(r"([\d+-]+)\s+(years?)", des, re.IGNORECASE)
            splitted_num = []
            for clipped in years_text:
                char_to_remove = ['+', '-', ' ']
                text_to_clean = clipped[0]

                for ch in char_to_remove:
                    for el in text_to_clean.split(ch):
                        if el.isalnum() is True:
                            splitted_num = splitted_num + [int(el)]

            return np.min(list(set(splitted_num)))

        if des_lan != 'en':
            translator = Translator()
            translation_dest = 'en'
            translation = translator.translate(job_des, dest=translation_dest)
            
            job_des = translation.text

        if job_des is True:
            return GetYears_RegEx(job_des)
        
        job_des = GetJobDescription(html_body)
        if job_des is True:
            return GetYears_RegEx(job_des)
    except:
        return None

    return None

# Get Company ID
def GetCompanyID(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['data']['companyDetails']['company']
        company_text = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['data', 'companyDetails', 'company'])
        return company_text.split('urn:li:fs_normalized_company:')[1]
    except:
        return None

# Get Company Name
def GetCompanyName(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['included'][1]['name']
        company_name = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-name', 'name'])
        if type(company_name) == str:
            return company_name
        
        return None
    except:
        return None

# Get Company Link
def GetCompanyLink(html_body):
    try:
        # soup = bs(html_body)
        # # id: bpr-guid-15912216
        # id_key = 'bpr-guid-15912216'
        # long_text = soup.find(id = id_key).text

        # json_dict = json.loads(long_text, strict = False)
        # return json_dict['included'][1]['url']
        company_link = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-url', 'url', 'list-primaryDescription', 'primaryDescription', 'attributesV2', 'list-detailData', 'detailData', 'hyperlink'])
        if type(company_link) == str:
            return company_link
        
        return None
    except:
        return None
    
# Get Company Top Card Details
# Example: ['11-50 employees', 'Software Development']
def GetCompanyTopCardDetails(html_body):
    soup = bs(html_body, 'lxml')
    # id: bpr-guid-15912217
    # id_key = 'bpr-guid-15912217'
    # long_text = soup.find(id = id_key).text

    # json_dict = json.loads(long_text, strict = False)
    # return json_dict['included'][4]['jobInsightsV2ResolutionResults'][1]['insightViewModel']['text']['text'].split(' · ')
    # dive_result = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults','list-insightViewModel', 'insightViewModel', 'text', 'text'], multi = True)
    # return dive_result
    try:
        dive_result = diveHtmlTags(html_body, toplevel_tag='code', dict_labels = ['included', 'list-jobInsightsV2ResolutionResults', 'jobInsightsV2ResolutionResults', 'list-insightViewModel', 'insightViewModel', 'text', 'text'], find_keyword = 'employees')
        if type(dive_result) == list:
            if dive_result[0].__contains__('works here') or dive_result[0].__contains__('work here'):
                return None
            elif dive_result[0].__contains__('hiring for this job'):
                return None
            elif dive_result[0].__contains__('Actively recruiting'):
                return None
        elif type(dive_result) == str:
            if dive_result.__contains__('works here') or dive_result.__contains__('work here'):
                return None
            elif dive_result.__contains__('hiring for this job'):
                return None
            elif dive_result.__contains__('Actively recruiting'):
                return None
            
        if type(dive_result) == str:
            if dive_result.__contains__('employees') == False:
                return None
        
        return dive_result.split(' · ')
    except:
        return None
    # for result in dive_result:
    #     print('r')
    #     print(result)
    #     print('--')
    #     if str.lower(result).__contains__('employee'):
    #         dive_result = result

    # return dive_result.split(' · ')

# Get Employee Count of Company
def GetCompanySize(html_body, details = []):
    try:
        if details is True:
            for detail in details:
                if 'employees' in detail:
                    return detail
                    
        details = GetCompanyTopCardDetails(html_body)
        for detail in details:
            if 'employees' in detail:
                return detail
    except:
        return None
    
    return None

# Get Industry that Company Specializes In
def GetCompanyIndustry(html_body, details = []):
    try:
        if details is True:
            for detail in details:
                if 'employees' not in detail:
                    return detail
                    
        details = GetCompanyTopCardDetails(html_body)
        for detail in details:
            if 'employees' not in detail:
                return detail
    except:
        return None
    
    return None

# Get Company Description
def GetCompanyDescription(html_body):
    try:
        soup = bs(html_body)

        long_text = soup.find('div', {'class': 'jobs-company__box'}).find('div', {'class': 'inline-show-more-text inline-show-more-text--is-collapsed inline-show-more-text--is-collapsed-with-line-clamp'}).text.strip()
        return long_text
    except:
        return None

In [9]:
def GetHTMLFromFile(file_path):
    try:
        f = open(file_path, "r")
        file_text = f.read()

        return file_text
    except:
        print(f'invalid file path: {file_path}')

def ReadFiles(files_df = pd.DataFrame([], columns = ['Job ID', 'page_path', 'skills_path', 'page_file_date', 'skills_file_date'])):
    job_files = []
    for file in glob.glob("data/*.txt"):
        job_files.append(file)

        job_id = 0
        f = open(file, "r")
        file_text = f.read()
        job_id = GetJobPostingID(file_text)
        if job_id == None:
            print(f'id not found in {file}')
            continue

        if job_id not in files_df['Job ID'].values:
            file_entry = {'Job ID':job_id, 'page_path':'', 'skills_path':''}
            files_df = pd.concat([files_df, pd.DataFrame([file_entry])], ignore_index=True)
            files_df.reset_index()

        if file.__contains__('skills.txt'):
            file_row = files_df[files_df['Job ID'] == job_id]
            file_row['skills_path'] = file
            # datetime.datetime.fromtimestamp(os.path.getctime("data/data_00.txt")).strftime('%Y-%m-%d')
            file_row['skills_file_date'] = os.path.getctime(file)

            files_df[files_df['Job ID'] == job_id] = file_row
        else:
            file_row = files_df[files_df['Job ID'] == job_id]
            file_row['page_path'] = file
            file_row['page_file_date'] = os.path.getctime(file)

            files_df[files_df['Job ID'] == job_id] = file_row
    
    return files_df

In [11]:
files_df_columns = ['Job ID', 'page_path', 'skills_path', 'page_file_date', 'skills_file_date']
files_df = pd.DataFrame([], columns = files_df_columns)

# Get the job post ID and file names of all .txt files in the /data folder
# Designating the file names to the identified post ID makes it easier to get job details
files_df = ReadFiles(files_df)

C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\1813570446.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_row['page_path'] = file
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\1813570446.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_row['page_file_date'] = os.path.getctime(file)
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\1813570446.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

id not found in data\data_157.txt
id not found in data\data_157_skills.txt
id not found in data\data_158.txt
id not found in data\data_158_skills.txt


C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\1813570446.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_row['page_path'] = file
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\1813570446.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_row['page_file_date'] = os.path.getctime(file)
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\1813570446.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [12]:
# Get details from .txt files and prepare dictionary to store in a dataframe
def GetJobDetails(html_body, html_skills_body):
    posting_id = GetJobPostingID(html_body)
    post_status = GetPostStatus(html_body)
    job_title = GetJobTitle(html_body)
    job_description = GetJobDescription(html_body)
    desc_lan = GetDescriptionLanguage(GetJobDescription(html_body))
    remote_status = GetRemoteWorkStatus(html_body)
    repost_status = GetRepostStatus(html_body)
    job_country = GetJobCountry(html_body)
    company_id = GetCompanyID(html_body)
    job_topcard_details = GetJobTopCardDetails(html_body)
    company_topcard_details = GetCompanyTopCardDetails(html_body)
    # company_topcard_details = []
    job_skills = GetJobSkills(html_skills_body)

    dict_job = {'Job ID': GetJobPostingID(html_body),
                'Post Date': '',
                'Status': GetPostStatus(html_body),
                'Job Link': GetPostLink(url_link=None, job_id=GetJobPostingID(html_body)),
                'Job Title': GetJobTitle(html_body),
                'Job Description': GetJobDescription(html_body),
                'Post Language': GetDescriptionLanguage(GetJobDescription(html_body)),
                'Remote': GetRemoteWorkStatus(html_body),
                'Reposted': GetRepostStatus(html_body),
                'Top Card Details': GetJobTopCardDetails(html_body),
                'Experience Level': GetJobExperienceLevel(html_body, details = job_topcard_details),
                'Experience in Years': GetJobYearsExperience(html_body, job_description, desc_lan),
                'Work Setting': GetJobWorkSetting(html_body, details = job_topcard_details),
                'Job Type': GetJobType(html_body, details = job_topcard_details),
                'Job Country': GetJobCountry(html_body),
                'Job Skills': GetJobSkills(html_skills_body),
                'Company ID': GetCompanyID(html_body),
                'Company Name': GetCompanyName(html_body),
                'Company Link': GetCompanyLink(html_body),
                'Company Card Details': GetCompanyTopCardDetails(html_body),
                'Company Size': GetCompanySize(html_body, details = company_topcard_details),
                'Industry': GetCompanyIndustry(html_body, details = company_topcard_details),
                'Company Description': GetCompanyDescription(html_body)
                }
    return dict_job

# ['Job ID', 'Post Date', 'Status', 'Job Link', 'Job Title', 'Job Description', 
# 'Post Language', 'Remote', 'Reposted', 'Top Card Details', 'Experience Level', 
# 'Experience in Years', 'Work Setting', 'Job Type', 'Job Country', 'Job Skills', 
# 'Company ID', 'Company Name', 'Company Link', 'Company Card Details', 'Company Size', 
# 'Industry', 'Company Description']
df_columns = ['Job ID', 'Post Date', 'Status', 'Job Link', 'Job Title', 'Job Description', 'Post Language', 'Remote', 'Reposted', 'Top Card Details', 'Experience Level', 'Experience in Years', 'Work Setting', 'Job Type', 'Job Country', 'Job Skills', 'Company ID', 'Company Name', 'Company Link', 'Company Card Details', 'Company Size', 'Industry', 'Company Description']

jobs_df = pd.DataFrame([], columns = df_columns)

# 
for index in files_df.index:
    html_body = GetHTMLFromFile(files_df.loc[index]['page_path'])
    html_skills = GetHTMLFromFile(files_df.loc[index]['skills_path'])

    job_entry = GetJobDetails(html_body, html_skills)
    job_entry['Post Date'] = list(files_df[files_df['Job ID'] == job_entry['Job ID']]['page_file_date'])[0]
    jobs_df = pd.concat([jobs_df, pd.DataFrame([job_entry])], ignore_index=True)
    jobs_df.reset_index()


Found "employees"
501-1,000 employees · Software Development
--
Found "employees"
501-1,000 employees · Software Development
--
Found "employees"
501-1,000 employees · Software Development
--
Found "employees"
501-1,000 employees · Software Development
--


C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\4113962444.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_jobs = pd.concat([df_jobs, pd.DataFrame([job_entry])], ignore_index=True)


Found "employees"
51-200 employees · Climate Data and Analytics
--
Found "employees"
51-200 employees · Climate Data and Analytics
--
Found "employees"
51-200 employees · Climate Data and Analytics
--
Found "employees"
51-200 employees · Climate Data and Analytics
--
Found "employees"
1-10 employees · Data Infrastructure and Analytics
--
Found "employees"
1-10 employees · Data Infrastructure and Analytics
--
Found "employees"
1-10 employees · Data Infrastructure and Analytics
--
Found "employees"
1-10 employees · Data Infrastructure and Analytics
--
Found "employees"
11-50 employees · Staffing and Recruiting
--
Found "employees"
11-50 employees · Staffing and Recruiting
--
Found "employees"
11-50 employees · Staffing and Recruiting
--
Found "employees"
11-50 employees · Staffing and Recruiting
--
Found "employees"
1,001-5,000 employees · IT Services and IT Consulting
--
Found "employees"
1,001-5,000 employees · IT Services and IT Consulting
--
Found "employees"
1,001-5,000 employees · 

In [13]:
jobs_df

,Job ID,Post Date,Status,Job Link,Job Title,Job Description,Post Language,Remote,Reposted,Top Card Details,...,Job Type,Job Country,Job Skills,Company ID,Company Name,Company Link,Company Card Details,Company Size,Industry,Company Description
0,3772602742,1.701295e+09,LISTED,https://www.linkedin.com/jobs/view/3772602742/,Financial Data Analyst,"About Team\n\nWe are Quadcode, a fintech compa...",en,True,None,"[Remote, Full-time, Mid-Senior level]",...,Full-time,EMEA,"[Analytical Skills, Communication, Attention t...",42345997,Quadcode,https://www.linkedin.com/company/quadcodecaree...,"[501-1,000 employees, Software Development]","501-1,000 employees",Software Development,Quadcode is a fintech company specializing in ...
1,3768710281,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3768710281/,Consultant in Data Analysis and Scientific Pro...,Your Responsibilities\n\nWe are currently look...,en,False,None,"[On-site, Full-time, Mid-Senior level]",...,Full-time,"Berlin, Berlin, Germany","[Communication, Data Analysis, Data Analytics,...",10198832,Climate Analytics,https://www.linkedin.com/company/climate-analy...,"[51-200 employees, Climate Data and Analytics]",51-200 employees,Climate Data and Analytics,Climate Analytics is a global climate science ...
2,3773349346,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3773349346/,Data-Analyst (m/w/d) in SaaS-Startup,"Wir sind cinify, ein junges Start-up, dass den...",de,True,None,"[Remote, Full-time, Entry level]",...,Full-time,Germany,"[Business Analysis, Dashboards, Data Analytics...",88277276,cinify,https://www.linkedin.com/company/cinify-io/life,"[1-10 employees, Data Infrastructure and Analy...",1-10 employees,Data Infrastructure and Analytics,cinify wurde Anfang des Jahres 2023 gegründet ...
3,3773351712,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3773351712/,"Data Engineer/Analyst - DBT, AWS, Redshift, Py...",Orcan Intelligence are currently partnered wit...,en,False,None,"[On-site, Contract]",...,Contract,"Berlin, Berlin, Germany","[Cloud Infrastructure, Data Engineering, Data ...",9387927,CareerAddict,https://www.linkedin.com/company/career-addict...,"[11-50 employees, Staffing and Recruiting]",11-50 employees,Staffing and Recruiting,CareerAddict is a leading career advice platfo...
4,3775291259,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3775291259/,Junior Business Intelligence Consultant (M/W/D),Wir sind die Positive Thinking Company. Wir su...,de,False,None,"[On-site, Full-time, Mid-Senior level]",...,Full-time,"Berlin, Berlin, Germany","[Analytics, Business Intelligence (BI), Data A...",11444799,Positive Thinking Company by CBTW,https://www.linkedin.com/company/the-positive-...,"[1,001-5,000 employees, IT Services and IT Con...","1,001-5,000 employees",IT Services and IT Consulting,𝐀𝐛𝐨𝐮𝐭 𝐏𝐨𝐬𝐢𝐭𝐢𝐯𝐞 𝐓𝐡𝐢𝐧𝐤𝐢𝐧𝐠 𝐂𝐨𝐦𝐩𝐚𝐧𝐲As the technolo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,3610427886,1.701465e+09,LISTED,https://www.linkedin.com/jobs/view/3610427886/,(Senior) Consultant Advanced Analytics (f/m/d),Strategy to Results.\n\nCAMELOT Management Con...,en,True,None,[],...,None,Germany,"[Analytics, Data Analytics, Data Mining, Data ...",89553,None,None,None,None,None,CAMELOT Management Consultants is the globally...
385,3765456123,1.701465e+09,LISTED,https://www.linkedin.com/jobs/view/3765456123/,Biostatistician - 100% remote - 12+ Months,Biostatistician - 100% remote - 12+ Months\nWe...,en,True,None,None,...,None,Germany,"[Biostatistics, Construction, EPCM, Health Tec...",100128089,None,None,None,None,None,Your Partner in Life Sciences Recruitment Acro...
386,3749323014,1.701465e+09,LISTED,https://www.linkedin.com/jobs/view/3749323014/,Quantitative Developer,Imagine a future where you employ your Modern ...,en,False,None,"[On-site, Full-time]",...,Full-time,"Gentofte, Capital Region, Denmark","[Data Science, Programming, Computer Science, ...",18324710,Ørsted,https://www.linkedin.com/company/orsted

In [15]:
for index in jobs_df.index:
    row = jobs_df.loc[index]

    if row['Company Card Details'] is not None:
        continue

    company_id = row['Company ID']
    company_details_df = jobs_df[(jobs_df['Company ID'] == company_id) & (jobs_df['Company Name'] is not None)]

    info_index = None
    for cd_index in company_details_df.index:
        if (company_details_df.loc[cd_index]['Company Name'] is not None) & (company_details_df.loc[cd_index]['Company Card Details'] is not None):
            info_index = cd_index
            break
    
    if info_index is None:
        continue

    if row['Company Name'] is None:
        row['Company Name'] = company_details_df.loc[info_index]['Company Name']
    if row['Company Card Details'] is None:
        row['Company Card Details'] = company_details_df.loc[info_index]['Company Card Details']
    if row['Company Size'] is None:
        row['Company Size'] = company_details_df.loc[info_index]['Company Size']
    if row['Industry'] is None:
        row['Industry'] = company_details_df.loc[info_index]['Industry']
    if row['Company Description'] is None:
        row['Company Description'] = company_details_df.loc[info_index]['Company Description']
    
    jobs_df.loc[index] = row

C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\4076228659.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Company Name'] = company_details_df.loc[info_index]['Company Name']
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\4076228659.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Company Card Details'] = company_details_df.loc[info_index]['Company Card Details']
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\4076228659.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [16]:
jobs_df.to_excel('linkedin_job_details.xlsx')  

In [17]:
top_skills_dict = {}

for keyword_skill in list(set().union(*jobs_df[pd.isna(jobs_df['Job Skills']) == False]['Job Skills'].values)):
    skills = []
    
    if keyword_skill.__contains__('('):
        skills = keyword_skill.split('(')

        skills = [skill.rstrip() for skill in skills]
        skills = [skill.replace(')', '') for skill in skills]
    else:
        skills = [keyword_skill]
    
    for skill in skills:
        if keyword_skill not in top_skills_dict.keys():
            top_skills_dict[keyword_skill] = skills
        else:
            top_skills_dict[keyword_skill] = list(set().union(top_skills_dict[keyword_skill], skills))
    
skills_columns = ['skill name', 'common phrases']

top_skills_df = pd.DataFrame([], columns=skills_columns)

for key in top_skills_dict.keys():
    dict_entry = {}
    dict_entry['skill name'] = key
    dict_entry['common phrases'] = top_skills_dict[key]
    top_skills_df = pd.concat([top_skills_df, pd.DataFrame([dict_entry])], ignore_index=True)
    top_skills_df.reset_index()

top_skills_df

,skill name,common phrases
0,Analytical Skills,[Analytical Skills]
1,Scanning Electron Microscopy (SEM),"[Scanning Electron Microscopy, SEM]"
2,SQL Server Analysis Services (SSAS),"[SSAS, SQL Server Analysis Services]"
3,Algorithms,[Algorithms]
4,Data Analysis,[Data Analysis]
...,...,...
793,Bank Confirmation,[Bank Confirmation]
794,Financial Modeling,[Financial Modeling]
795,Fraud Investigations,[Fraud Investigations]
796,Client-focused,[Client-focused]


In [18]:
jobs_df['Associated Skill Phrases'] = ''
jobs_df['Unlisted Skill'] = ''
jobs_df['Unlisted Skill Phrases'] = ''

def get_listed_skill_phrases(row, skills_df):
    associated_skills = []

    if row['Job Skills'] is None:
        return row

    for skill in row['Job Skills']:
        # print(skill)
        # print(top_skills_df[top_skills_df['skill name'] == skill]['common phrases'].values)
        # print(associated_skills)
        associated_skills.append(top_skills_df[top_skills_df['skill name'] == skill]['common phrases'].tolist()[0])
        # associated_skills.append(top_skills_df[top_skills_df['skill name'] == skill]['common phrases'].values[0])
        # associated_skills = associated_skills + top_skills_df[top_skills_df['skill name'] == skill]['common phrases'].values
    row['Associated Skill Phrases'] = list(set().union(*associated_skills))
   
    return row
    
def get_unlisted_skill_phrases(row, skills_df):
    unlisted_skills = []
    unlisted_skill_phrases = []

    for index in skills_df.index:
        if row['Job Skills'] is not None:
            if skills_df.loc[index]['skill name'] in row['Job Skills']:
                continue
        
        for phrase in skills_df.loc[index]['common phrases']:
            if phrase in row['Job Description'].split(' '):
                unlisted_skills.append(skills_df.loc[index]['skill name'])
                unlisted_skill_phrases.append(phrase)
    
    row['Unlisted Skill'] = unlisted_skills
    row['Unlisted Skill Phrases'] = unlisted_skill_phrases

    return row

for index in jobs_df.index:
    skill_row = jobs_df.loc[index]
    jobs_df.loc[index] = get_listed_skill_phrases(skill_row, top_skills_df)
    jobs_df.loc[index] = get_unlisted_skill_phrases(skill_row, top_skills_df)


jobs_df.head(5)

C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\2329831534.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Associated Skill Phrases'] = list(set().union(*associated_skills))
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\2329831534.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['Unlisted Skill'] = unlisted_skills
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\2329831534.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

,Job ID,Post Date,Status,Job Link,Job Title,Job Description,Post Language,Remote,Reposted,Top Card Details,...,Company ID,Company Name,Company Link,Company Card Details,Company Size,Industry,Company Description,Associated Skill Phrases,Unlisted Skill,Unlisted Skill Phrases
0,3772602742,1.701295e+09,LISTED,https://www.linkedin.com/jobs/view/3772602742/,Financial Data Analyst,"About Team\n\nWe are Quadcode, a fintech compa...",en,True,None,"[Remote, Full-time, Mid-Senior level]",...,42345997,Quadcode,https://www.linkedin.com/company/quadcodecaree...,"[501-1,000 employees, Software Development]","501-1,000 employees",Software Development,Quadcode is a fintech company specializing in ...,"[Analytical Skills, English, B2, Quantitative ...","[SQL, Python (Programming Language), Software ...","[SQL, Python, SaaS]"
1,3768710281,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3768710281/,Consultant in Data Analysis and Scientific Pro...,Your Responsibilities\n\nWe are currently look...,en,False,None,"[On-site, Full-time, Mid-Senior level]",...,10198832,Climate Analytics,https://www.linkedin.com/company/climate-analy...,"[51-200 employees, Climate Data and Analytics]",51-200 employees,Climate Data and Analytics,Climate Analytics is a global climate science ...,"[Data Visualization, Data Analysis, Data Analy...","[Java, Analytics]","[Java, Analytics]"
2,3773349346,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3773349346/,Data-Analyst (m/w/d) in SaaS-Startup,"Wir sind cinify, ein junges Start-up, dass den...",de,True,None,"[Remote, Full-time, Entry level]",...,88277276,cinify,https://www.linkedin.com/company/cinify-io/life,"[1-10 employees, Data Infrastructure and Analy...",1-10 employees,Data Infrastructure and Analytics,cinify wurde Anfang des Jahres 2023 gegründet ...,"[Extract, Transform, Load, Data Visualization,...","[Business Intelligence (BI), SAS (Software), H...","[BI, Software, Software, Software, Software]"
3,3773351712,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3773351712/,"Data Engineer/Analyst - DBT, AWS, Redshift, Py...",Orcan Intelligence are currently partnered wit...,en,False,None,"[On-site, Contract]",...,9387927,CareerAddict,https://www.linkedin.com/company/career-addict...,"[11-50 employees, Staffing and Recruiting]",11-50 employees,Staffing and Recruiting,CareerAddict is a leading career advice platfo...,"[Extract, Transform, Load, Data Science, Data ...",[],[]
4,3775291259,1.701296e+09,LISTED,https://www.linkedin.com/jobs/view/3775291259/,Junior Business Intelligence Consultant (M/W/D),Wir sind die Positive Thinking Company. Wir su...,de,False,None,"[On-site, Full-time, Mid-Senior level]",...,11444799,Positive Thinking Company by CBTW,https://www.linkedin.com/company/the-positive-...,"[1,001-5,000 employees, IT Services and IT Con...","1,001-5,000 employees",IT Services and IT Consulting,𝐀𝐛𝐨𝐮𝐭 𝐏𝐨𝐬𝐢𝐭𝐢𝐯𝐞 𝐓𝐡𝐢𝐧𝐤𝐢𝐧𝐠 𝐂𝐨𝐦𝐩𝐚𝐧𝐲As the technolo...,"[SAP, BPCS, Data Analytics, SAP BPC, IBM Plann...",[],[]


In [19]:
jobs_df.to_excel('linkedin_job_details.xlsx')  

In [20]:
top_skills_df['Listed frequency'] = 0
top_skills_df['Unlisted frequency'] = 0

def group_list(lst):
     
    return list(zip(Counter(lst).keys(), Counter(lst).values()))
     
combined_listed_skill_lists = list(itertools.chain.from_iterable(jobs_df[pd.isna(jobs_df['Job Skills']) == False]['Job Skills'].values))
combined_unlisted_skill_lists = list(itertools.chain.from_iterable(jobs_df['Unlisted Skill'].values))

listed_skills_grouped = group_list(combined_listed_skill_lists)
unlisted_skills_grouped = group_list(combined_unlisted_skill_lists)

for skill_tuple in listed_skills_grouped:
    skill_row = top_skills_df[top_skills_df['skill name'] == skill_tuple[0]]

    skill_row['Listed frequency'] = skill_tuple[1]
    top_skills_df.loc[skill_row.index] = skill_row


for skill_tuple in unlisted_skills_grouped:
    skill_row = top_skills_df[top_skills_df['skill name'] == skill_tuple[0]]

    skill_row['Unlisted frequency'] = skill_tuple[1]
    top_skills_df.loc[skill_row.index] = skill_row

top_skills_df

C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\2873718512.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skill_row['Listed frequency'] = skill_tuple[1]
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\2873718512.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skill_row['Unlisted frequency'] = skill_tuple[1]


,skill name,common phrases,Listed frequency,Unlisted frequency
0,Analytical Skills,[Analytical Skills],73,0
1,Scanning Electron Microscopy (SEM),"[Scanning Electron Microscopy, SEM]",1,0
2,SQL Server Analysis Services (SSAS),"[SSAS, SQL Server Analysis Services]",1,0
3,Algorithms,[Algorithms],6,0
4,Data Analysis,[Data Analysis],55,0
...,...,...,...,...
793,Bank Confirmation,[Bank Confirmation],1,0
794,Financial Modeling,[Financial Modeling],3,0
795,Fraud Investigations,[Fraud Investigations],2,0
796,Client-focused,[Client-focused],1,0


In [21]:
# init the Google API translator
translator = Translator()
translation_dest = 'en'

In [22]:
df_translated = jobs_df.copy()
df_translated = df_translated[['Job ID', 'Job Title', 'Job Description', 'Post Language', 'Remote', 'Top Card Details', 'Job Country', 'Job Skills', 'Company Name', 'Company Card Details', 'Industry', 'Company Description']]

def translate_phrases(phrase_to_translate, translate_dest = 'en'):
    if type(phrase_to_translate) != str:
        return phrase_to_translate
    
    translation = translator.translate(phrase_to_translate, dest=translate_dest)
    return translation.text

df_translated['final'] = ''
for index in df_translated.index:
    row = df_translated.loc[index]
    if row['Post Language'] == translation_dest:
        row['final'] = ', '.join([str(el) for el in df_translated.loc[index].tolist()])
        if row['Remote'] == True:
            row['final'] = row['final'] + ', Remote'
    else:
        row['final'] = ', '.join([str(translate_phrases(el, translation_dest)) for el in df_translated.loc[index].tolist()])
        if row['Remote'] == True:
            row['final'] = row['final'] + ', Remote'
    
    df_translated.loc[index] = row

jobs_df['translated_en'] = df_translated['final']

C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\358001166.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['final'] = ', '.join([str(el) for el in df_translated.loc[index].tolist()])
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\358001166.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['final'] = row['final'] + ', Remote'
C:\Users\naimi\AppData\Local\Temp\ipykernel_26372\358001166.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [24]:
jobs_df.to_excel('linkedin_job_details.xlsx') 
top_skills_df.to_excel('linkedin_job_skills.xlsx') 